In [6]:
import os

# Key

In [2]:
BIGQUERY_CREDENTIALS_FILE_PATH = r"/Users/taylorlin/Desktop/GCP_Key/harry_GCS_BigQuery_write_cred.json"
# BIGQUERY_CREDENTIALS_FILE_PATH = r"D:\data_engineer\TIR_group2\TIR101_Group2\secrets\harry_GCS_BigQuery_write_cred.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = BIGQUERY_CREDENTIALS_FILE_PATH

In [3]:
from google.cloud import bigquery

In [4]:
BQ_CLIENT = bigquery.Client()  

In [5]:
# showing project name(level higher than dataset)
BQ_CLIENT.project

'tir101-group2-422603'

# Dataset Creation in BQ

In [7]:
def create_dataset(dataset_name:str , location:str="asia-east1",client: bigquery.Client = BQ_CLIENT):
    """Create a dataset"""
    porject_name = client.project
    dataset_id = f"{porject_name}.{dataset_name}"

    dataset = bigquery.Dataset(dataset_id)
    dataset.location = location

    client.create_dataset(dataset)
    print(f"created {dataset_name} in project {porject_name} ,location:{location}")

In [8]:
create_dataset(dataset_name="BUS_GCS_to_BQ_SRC_ODS_DIM")

created BUS_GCS_to_BQ_SRC_ODS_DIM in project tir101-group2-422603 ,location:asia-east1


bus_station_id,station_name,address,city_code,district,lat,lng,bearing,subarea

# GCS-CSV to BQ-SRC-CSV

In [10]:
def GCS_tpe_bus_static_to_BQ_SRC(client: bigquery.Client = BQ_CLIENT) -> None:
    """create external table (GCS to BQ) for bus raw data(tpe_bus_station_info_after_api_call)"""
    query_job = client.query(
    """
        CREATE OR REPLACE EXTERNAL TABLE `BUS_GCS_to_BQ_SRC_ODS_DIM.SRC_Bus_static_data`
        (
            `bus_station_id` INT,
            `station_name` STRING,
            `address` STRING,
            `city_code` STRING,
            `district` STRING,
            `lat` FLOAT64,
            `lng` FLOAT64,
            `bearing` STRING,
            `subarea` STRING
        )
        OPTIONS 
        (
            format = 'CSV',
            uris = ['gs://static_reference/bus/tpe_bus_station_info_after_api_call.csv'],
            skip_leading_rows = 1,
            max_bad_records = 1
        )
    """
    )
    query_job.result()
    print("SRC_Bus_static_data is created.")
GCS_tpe_bus_static_to_BQ_SRC()

SRC_Bus_static_data is created.


# BQ-SRC transforming to BQ-ODS

刪除 `subarea` 欄位
新增
  `create_time` timestamp,
  `update_time` timestamp


CREATE TABLE `bus_station` (
  `bus_station_id` int PRIMARY KEY,
  `station_name` varchar(20),
  `address` varchar(50),
  `city_code` char(3),
  `district` varchar(10),
  `lat` decimal(8,6),
  `lng` decimal(9,6),
  `bearing` char(1),
  `create_time` timestamp,
  `update_time` timestamp
);

In [12]:
def BQ_SRC_to_ODS_bus_station(client: bigquery.Client = BQ_CLIENT) -> None:
    """Transforming the SRC layer of bus stations into the ODS layer of bus stations."""
    query_job = client.query(
    """
    CREATE OR REPLACE TABLE `BUS_GCS_to_BQ_SRC_ODS_DIM.ODS_Bus_static_data` AS
    SELECT
        `bus_station_id`,
        TRIM(`station_name`) AS `station_name`,
        TRIM(`address`) AS `address`,
        TRIM(`city_code`) AS `city_code`,
        TRIM(`district`) AS `district`,
        `lat`,
        `lng`,
        TRIM(`bearing`) AS `bearing`,
        CURRENT_TIMESTAMP() AS update_time
    from 
        BUS_GCS_to_BQ_SRC_ODS_DIM.SRC_Bus_static_data
    """
    )
    query_job.result()
    print("ODS_Bus_static_data is created")
BQ_SRC_to_ODS_bus_station()

ODS_Bus_static_data is created


In [14]:
def BQ_ODS_to_DIM_bus_station(client: bigquery.Client = BQ_CLIENT) -> None:
    """Transforming the SRC layer of bus stations into the ODS layer of bus stations."""
    query_job = client.query(
    """
    CREATE OR REPLACE TABLE `BUS_GCS_to_BQ_SRC_ODS_DIM.DIM_Bus_static_data` AS
    SELECT *
    from 
        BUS_GCS_to_BQ_SRC_ODS_DIM.ODS_Bus_static_data
    """
    )
    query_job.result()
    print("DIM_Bus_static_data is created")
BQ_ODS_to_DIM_bus_station()

DIM_Bus_static_data is created
